In [ ]:
import ee
import numpy as np
import rasterio
import os
from glob import glob
from tqdm import tqdm
from utils.functions import get_files_info
GEE_PROJECT_ID = "fusion-371234"
ee.Authenticate()
ee.Initialize(project=GEE_PROJECT_ID)

In [14]:
n_files, name_files = get_files_info("training")
name_files = sorted(name_files)

print(f"Number of Images per Date: {n_files//10}")

Total number of files: 240
Number of Images per Date: 24


In [19]:
def vectorize_sentinel_256(file_path):
    with rasterio.open(file_path) as src:
        img = src.read()  # [12, 256, 256]
    
    bands, H, W = img.shape
    img = np.transpose(img, (1, 2, 0))  # [H, W, B]
    X = img.reshape(-1, bands)          # [65536, 12]

    return X

def load_training_images(folder):
    files = sorted(glob(os.path.join(folder, "*.tif")))
    print(f"Encontradas {len(files)} imagens.")

    X_list = []
    for f in tqdm(files, desc="Carregando dataset"):
        X_list.append(vectorize_sentinel_256(f))

    # Empilha tudo → [N_imagens*65536, 12]
    X_all = np.vstack(X_list)
    print("Shape final do dataset:", X_all.shape)
    return X_all

def fit_baseline_gaussian(X):
    mu = X.mean(axis=0)
    cov = np.cov(X, rowvar=False)
    cov_inv = np.linalg.pinv(cov)  
    return mu, cov_inv

# 1 — Treinar baseline
X_train = load_training_images("training")  
mu, cov_inv = fit_baseline_gaussian(X_train)


Encontradas 240 imagens.


Carregando dataset: 100%|██████████| 240/240 [00:01<00:00, 189.10it/s]


Shape final do dataset: (15728640, 12)
